Spark SQL is module in PySpark which is used to process the Structured data. This allows to integrate SQL queries with the spark program  and allows to process in distrbuted and fault tolorent manner.

Running SQL-like queries in PySpark constists several steps:

* Start by initializing a SparkSession. This is the entry point to PySpark.
* Load your data into a DataFrame.
* Register the DataFrame as a temporary view so that you can query it using SQL,where we can run our SQL Queries with spark.sql() method
* Process the results returned by the SQL query. You can further manipulate the results as needed.

# SQL operations illustrated in this notebook
Before illustraing the PySpark Sql operations, how join operations in PySpark is perfromed is shown.
* select
* filter
* order by
* group by
* SQL Joins
   - inner join
   - left join
   - right join
   - full outer join
   - cross join




In [ ]:
# import necessary liberaries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
# creating the spark session
spark=SparkSession.builder.appName('sparkSQL').getOrCreate()
spark

In [7]:
# loading the data from csv file
uber_data=spark.read.csv('uber_data.csv',header=True,inferSchema=True)
uber_data.show(10)
uber_data.printSchema()

+---------+------------+---------+-------+----------------+---------+--------------+
|     Date|Time (Local)|Eyeballs |Zeroes |Completed Trips |Requests |Unique Drivers|
+---------+------------+---------+-------+----------------+---------+--------------+
|10-Sep-12|           7|        5|      0|               2|        2|             9|
|     NULL|           8|        6|      0|               2|        2|            14|
|     NULL|           9|        8|      3|               0|        0|            14|
|     NULL|          10|        9|      2|               0|        1|            14|
|     NULL|          11|       11|      1|               4|        4|            11|
|     NULL|          12|       12|      0|               2|        2|            11|
|     NULL|          13|        9|      1|               0|        0|             9|
|     NULL|          14|       12|      1|               0|        0|             9|
|     NULL|          15|       11|      2|               1|      

In [6]:
# create tempurary view for executing SQL queries
uber_data.createOrReplaceTempView('rw_uber')

In [8]:
# check if the tempuraty table is created
spark.catalog.listTables()

[Table(name='rw_uber', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [12]:
# selecting from spark dataframe
uber_data.select('Date','Time (Local)','Eyeballs ').show(5)

+---------+------------+---------+
|     Date|Time (Local)|Eyeballs |
+---------+------------+---------+
|10-Sep-12|           7|        5|
|     NULL|           8|        6|
|     NULL|           9|        8|
|     NULL|          10|        9|
|     NULL|          11|       11|
+---------+------------+---------+
only showing top 5 rows



In [13]:
# performing the same operation with the spark sql
spark.sql('select `Date`,`Time (Local)`,`Eyeballs ` from rw_uber').show(5)

+---------+------------+---------+
|     Date|Time (Local)|Eyeballs |
+---------+------------+---------+
|10-Sep-12|           7|        5|
|     NULL|           8|        6|
|     NULL|           9|        8|
|     NULL|          10|        9|
|     NULL|          11|       11|
+---------+------------+---------+
only showing top 5 rows



In [20]:
# filtering the records
spark.sql('select * from rw_uber') \
      .where(" 'Eyeballs'>='1'")\
      .show(5)

+---------+------------+---------+-------+----------------+---------+--------------+
|     Date|Time (Local)|Eyeballs |Zeroes |Completed Trips |Requests |Unique Drivers|
+---------+------------+---------+-------+----------------+---------+--------------+
|10-Sep-12|           7|        5|      0|               2|        2|             9|
|     NULL|           8|        6|      0|               2|        2|            14|
|     NULL|           9|        8|      3|               0|        0|            14|
|     NULL|          10|        9|      2|               0|        1|            14|
|     NULL|          11|       11|      1|               4|        4|            11|
+---------+------------+---------+-------+----------------+---------+--------------+
only showing top 5 rows



In [21]:
# sorting the records
spark.sql('select * from rw_uber') \
      .where(" 'Eyeballs'>='1'")\
      .orderBy('Date')\
      .show(5)

+----+------------+---------+-------+----------------+---------+--------------+
|Date|Time (Local)|Eyeballs |Zeroes |Completed Trips |Requests |Unique Drivers|
+----+------------+---------+-------+----------------+---------+--------------+
|NULL|          13|        9|      1|               0|        0|             9|
|NULL|          18|       11|      1|               3|        4|             7|
|NULL|          14|       12|      1|               0|        0|             9|
|NULL|           8|        6|      0|               2|        2|            14|
|NULL|          15|       11|      2|               1|        2|             7|
+----+------------+---------+-------+----------------+---------+--------------+
only showing top 5 rows



In [29]:
spark.sql("select * from rw_uber where 'Eyeballs' >='1' order by 'Eyeballs' limit 5 ") \
     .show()


+---------+------------+---------+-------+----------------+---------+--------------+
|     Date|Time (Local)|Eyeballs |Zeroes |Completed Trips |Requests |Unique Drivers|
+---------+------------+---------+-------+----------------+---------+--------------+
|10-Sep-12|           7|        5|      0|               2|        2|             9|
|     NULL|           8|        6|      0|               2|        2|            14|
|     NULL|           9|        8|      3|               0|        0|            14|
|     NULL|          10|        9|      2|               0|        1|            14|
|     NULL|          11|       11|      1|               4|        4|            11|
+---------+------------+---------+-------+----------------+---------+--------------+



In [30]:
# group by
uber_data.groupBy("Eyeballs ").count() \
  .show()

+---------+-----+
|Eyeballs |count|
+---------+-----+
|       31|    3|
|       53|    2|
|       34|    6|
|       28|    5|
|       26|    5|
|       27|    2|
|       44|    1|
|       12|    7|
|       22|    9|
|       47|    1|
|        1|   19|
|       13|    6|
|        6|    8|
|       16|   10|
|        3|   14|
|       20|   12|
|       40|    2|
|       94|    1|
|        5|   11|
|       19|   14|
+---------+-----+
only showing top 20 rows



In [31]:
spark.sql("select `Eyeballs `,count(*) from rw_uber group by `Eyeballs `").show()

+---------+--------+
|Eyeballs |count(1)|
+---------+--------+
|       31|       3|
|       53|       2|
|       34|       6|
|       28|       5|
|       26|       5|
|       27|       2|
|       44|       1|
|       12|       7|
|       22|       9|
|       47|       1|
|        1|      19|
|       13|       6|
|        6|       8|
|       16|      10|
|        3|      14|
|       20|      12|
|       40|       2|
|       94|       1|
|        5|      11|
|       19|      14|
+---------+--------+
only showing top 20 rows



**PySpark joins**  are used to combine two or more that two spark dataframes. I provides all the basic join operations that are avaliable in SQL.
These are the wider transformations that involves the data suffeling accross the network.
However, **PySpark sql** are more optimized by default.Although PySpark sql has some performance issue, nderstanding how to effectively utilize PySpark joins is essential for conducting comprehensive data analysis, building data pipelines, and deriving valuable insights from large-scale datasets.

### pyapark Join syntax:
**join(self, other, on=None, how=None)**

**join()** function is used to join two dataframes, where **other** is the data frame which you want to join with, **on** is the name of the field with wiich table will be joined, **how**  means type of the join that you want to perfrom.

We can also write join expresssion by adding where() and filter() methods on dataframe and can have join operation on the multiple columns.

In [32]:
# creating dummy dataframe
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [33]:
dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]

deptColumns = ["dept_name","dept_id"]

deptDF = spark.createDataFrame(data=dept, schema = deptColumns)

deptDF.printSchema()

deptDF.show(truncate=False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [34]:
# PySpark Inner join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"inner") \
     .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [35]:
# PySpark Left Outer join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"left") \
      .show(truncate=False)
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftouter") \
    .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|6     |Brown   |2              |2010       |50         |      |-1    |NULL     |NULL   |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|n

In [36]:
#PySpark right outer join:
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"right") \
   .show(truncate=False)
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"rightouter") \
   .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|NULL  |NULL    |NULL           |NULL       |NULL       |NULL  |NULL  |Sales    |30     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|n

In [37]:
# PySpark full outer join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"outer") \
    .show(truncate=False)
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"full") \
    .show(truncate=False)
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"fullouter") \
    .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|NULL  |NULL    |NULL           |NULL       |NULL       |NULL  |NULL  |Sales    |30     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
|6     |Brown   |2              |2010       |50         |      |-1    |NULL     |NULL   |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+-

In [38]:
# PySpark left semi join
'''
lest semi join returns only the rows in left table for which there is a match in the right table.
'''
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftsemi") \
   .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
+------+--------+---------------+-----------+-----------+------+------+



In [39]:
# PySpark left anti join
'''
lest semi join returns only the rows in left table for which there is  no  match in the right table.
'''
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftanti") \
   .show(truncate=False)

+------+-----+---------------+-----------+-----------+------+------+
|emp_id|name |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|6     |Brown|2              |2010       |50         |      |-1    |
+------+-----+---------------+-----------+-----------+------+------+



In [41]:
# PySpark self join
'''joining table with itself'''
empDF.alias("emp1").join(empDF.alias("emp2"), \
    col("emp1.superior_emp_id") == col("emp2.emp_id"),"inner") \
    .select(col("emp1.emp_id"),col("emp1.name"), \
      col("emp2.emp_id").alias("superior_emp_id"), \
      col("emp2.name").alias("superior_emp_name")) \
   .show(truncate=False)

+------+--------+---------------+-----------------+
|emp_id|name    |superior_emp_id|superior_emp_name|
+------+--------+---------------+-----------------+
|2     |Rose    |1              |Smith            |
|3     |Williams|1              |Smith            |
|4     |Jones   |2              |Rose             |
|5     |Brown   |2              |Rose             |
|6     |Brown   |2              |Rose             |
+------+--------+---------------+-----------------+



# PySpark Sql Joins
All above mentions join operations can alternatively performed by using spark SQl. For this create a temporary view using createOrReplaceTempView(), then use the spark.sql() to execute the join query.

In [43]:
# creating tempurary views
empDF.createOrReplaceTempView("EMP")
deptDF.createOrReplaceTempView("DEPT")

# inner join
joinDF2 = spark.sql("select * from EMP e INNER JOIN DEPT d ON e.emp_dept_id == d.dept_id") \
  .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [46]:
# left outer join
joinDF2 = spark.sql("select * from EMP e LEFT JOIN DEPT d ON e.emp_dept_id = d.dept_id") \
  .show(truncate=False)


+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|6     |Brown   |2              |2010       |50         |      |-1    |NULL     |NULL   |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [59]:
# full outer join
joinDF2 = spark.sql("select * from EMP e FULL JOIN DEPT d ON e.emp_dept_id = d.dept_id") \
  .show(truncate=False)


+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|NULL  |NULL    |NULL           |NULL       |NULL       |NULL  |NULL  |Sales    |30     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
|6     |Brown   |2              |2010       |50         |      |-1    |NULL     |NULL   |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [58]:
# right outer join
joinDF2 = spark.sql("select * from EMP e RIGHT JOIN DEPT d ON e.emp_dept_id = d.dept_id") \
  .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|NULL  |NULL    |NULL           |NULL       |NULL       |NULL  |NULL  |Sales    |30     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [57]:
# left semi join
joinDF2 = spark.sql("select * from EMP e LEFT JOIN DEPT d ON e.emp_dept_id = d.dept_id where e.emp_dept_id==d.dept_id ") \
  .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [56]:
# anti left join
joinDF2 = spark.sql("select * from EMP e LEFT JOIN DEPT d ON e.emp_dept_id = d.dept_id where e.emp_dept_id != d.dept_id ") \
  .show(truncate=False)

+------+----+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+----+---------------+-----------+-----------+------+------+---------+-------+
+------+----+---------------+-----------+-----------+------+------+---------+-------+



In [60]:
# self join
spark.sql("""
  SELECT emp1.emp_id, emp1.name,
         emp2.emp_id AS superior_emp_id,
         emp2.name AS superior_emp_name
  FROM EMP emp1
  INNER JOIN EMP emp2
    ON emp1.superior_emp_id = emp2.emp_id
""").show(truncate=False)


+------+--------+---------------+-----------------+
|emp_id|name    |superior_emp_id|superior_emp_name|
+------+--------+---------------+-----------------+
|2     |Rose    |1              |Smith            |
|3     |Williams|1              |Smith            |
|4     |Jones   |2              |Rose             |
|5     |Brown   |2              |Rose             |
|6     |Brown   |2              |Rose             |
+------+--------+---------------+-----------------+

